In [18]:
import tensorflow as tf
from PIL import Image
import numpy as np
from tensorflow_addons.optimizers import AdamW
import pandas as pd

In [15]:
export_dir = "../malaria/ml_logic/vit_classifier"
model = tf.keras.models.load_model(
    export_dir, custom_objects={"AdamW":AdamW}
    )

In [238]:
i,j = 95,5995
img_array = np.load("../malaria/train_test_data/200images/training_COL_X_21778.npy")
shp = img_array.shape
img = img_array[i:j].reshape(-1,200,200,3)

print(f"Number of samples to analyse:{img.shape}")
target_array = np.load("../malaria/train_test_data/200images/training_COL_y_21778.npy")
target_df = pd.DataFrame(target_array[i:j])

Number of samples to analyse:(5900, 200, 200, 3)


In [239]:
target_df.value_counts()

red blood cell    4217
trophozoite       1134
ring               303
schizont           129
gametocyte         117
dtype: int64

In [240]:
pred = model(img)
pred

<tf.Tensor: shape=(5900, 4), dtype=float32, numpy=
array([[-4.5744576 ,  1.3910407 , -2.7799022 ,  4.908291  ],
       [-3.6262376 ,  1.2513802 , -3.3928077 ,  4.231205  ],
       [-8.360654  ,  2.0223699 , -5.955366  ,  6.1781383 ],
       ...,
       [-2.1452513 ,  0.76642126, -2.5850797 ,  2.7817326 ],
       [-3.9495637 ,  3.5811696 , -3.9457862 ,  3.159606  ],
       [-6.6826777 ,  6.9782314 , -6.6645927 ,  4.6405573 ]],
      dtype=float32)>

In [241]:
output= (tf.math.argmax(pred, axis = 1))
output_df = pd.DataFrame(output, columns = ["VIT_Type"])

In [269]:
le_VIT = {0:"gametocyte",
          1:"ring",
          2:"schizont",
          3:"trophozoite"}
output_df["VIT_Type"].replace(le_VIT, inplace=True)
infect_results_df = pd.DataFrame(output_df.value_counts())

## This is for testing results set

In [245]:
comb_df = pd.concat([target_df, output_df], axis = 1)
comb_df["diff"]=np.where(comb_df["Histol_Type"]!=comb_df["VIT_Type"], True, False)

In [246]:
comb_df[comb_df["diff"]==False]

,Histol_Type,VIT_Type,diff
0,trophozoite,trophozoite,False
1,trophozoite,trophozoite,False
2,trophozoite,trophozoite,False
3,trophozoite,trophozoite,False
4,trophozoite,trophozoite,False
...,...,...,...
1678,trophozoite,trophozoite,False
1679,trophozoite,trophozoite,False
1680,trophozoite,trophozoite,False
1681,trophozoite,trophozoite,False


In [260]:
pd.set_option('display.max_rows',comb_df.shape[0]+1)
pd.DataFrame(comb_df.value_counts())

0
Histol_Type    VIT_Type    diff       
red blood cell ring        True   2692
               trophozoite True   1523
trophozoite    trophozoite False  1122
ring           trophozoite True    154
               ring        False   149
gametocyte     trophozoite True    115
schizont       trophozoite True     93
               schizont    False    36
trophozoite    schizont    True      9
               ring        True      3
red blood cell schizont    True      2
gametocyte     gametocyte  False     1
               schizont    True      1